In [46]:
import os
import datetime
import copy
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

import Tracab as tracab
import Tracking_Visuals as vis
import Tracking_Velocities as vel
import Tracking_Fatigue as fatigue
import helpers

import importlib
importlib.reload(tracab)
importlib.reload(helpers)
importlib.reload(vis)

<module 'Tracking_Visuals' from '/Users/jeffbalkanski/research_soccer/SoccerTrackingData3/Tracking_Visuals.py'>

In [3]:
# # config
# current_dir = os.path.dirname(os.getcwd())
# dir_path = os.path.join(current_dir, 'Aalborg_Jeff') # path to directory of Tracab data
# LEAGUE = 'DSL'

# fname = '984455'
# match_id = fname

In [5]:
# # read
# fpath = os.path.join(dir_path, fname, fname + '_TracDAT')
# frames_tb, match_tb, team1_players, team0_players = tracab.read_tracab_match_data(LEAGUE, fpath, fname, verbose=True)

/Users/jeffbalkanski/research_soccer/Aalborg_Jeff/984455/984455_TracDAT
Reading match metadata
Reading match tracking data
Timestamping frames
Measuring velocities
home goalkeeper(s):  [28]
away goalkeeper(s):  [1]
0 70642
70643 142733


## Read all

In [47]:
# read all data
with open(os.path.join('./saved', 'all_time_series.pkl'), 'rb') as infile:
    data_dict = pickle.load(infile)

# Half time analysis

In [48]:
# one game
match_id = '984455'
EX_mat = data_dict[match_id]['energy_x']

# half time frame
end_first = data_dict[match_id]['game_info']['last_frame_first_half']

In [49]:
split = end_first
k_frames = 20000
halftime_info = []


In [50]:
# for each player
for _, player_id, series in EX_mat:
    # make cumulative EX
    series = np.cumsum(series)
    
    # split
    first, second = series[:split], series[split:]
    xs = np.array(range(k_frames))
    
    # first
    slope_1, intercept_1 = np.polyfit(xs, first[-k_frames:], 1)
    slope_2, intercept_2 = np.polyfit(xs, second[:k_frames], 1)
    halftime_info.append([match_id, player_id, split, slope_1, slope_2])
    
halftime_df = pd.DataFrame(halftime_info, columns=['match_id', 'player_id', 'split', 
                                                'slope_before', 'slope_after'])

In [51]:
first

array([2.4000000e-01, 4.8000000e-01, 7.2000000e-01, ..., 6.9192140e+05,
       6.9192724e+05, 6.9193041e+05])

In [52]:
for _, playerid, _,  slope_before, slope_after in halftime_df.values:
    print(slope_after - slope_before)

2.1190446721902774
0.10922723676007884
-1.2319065040370472
-1.7139953088840407
-1.0901810475883185
-1.1576651870268684
4.1805060225586566
-1.4683080653135274
-0.33222441520161006
1.1514763418496656
2.897224627476845
-1.5092933469370906
2.113279834045745
1.3418672419691973
-0.6390099048140883
-6.4791772890236
0.54020389334047
-3.4071270349238727
2.4585143645997487
-0.6043887435448632
-2.0853592112214603
3.974334393838321


In [45]:
for player, data in data_dict.items():
    data['energy']